<b>Partie IA</b>

Imports

In [45]:
# Importation des bibliothèques nécessaires
import pandas as pd  # Pour manipuler les données sous forme de DataFrame
import warnings  # Pour gérer les avertissements

# Importation des modules de Scikit-learn pour la transformation des colonnes, pipelines, modèles et évaluation
from sklearn.compose import ColumnTransformer  # Pour appliquer des transformations différentes sur les colonnes
from sklearn.pipeline import Pipeline  # Pour créer des pipelines de transformations et de modèles
from sklearn.preprocessing import StandardScaler  # Pour le prétraitement des données
from sklearn.model_selection import train_test_split  # Pour la division des données et la recherche des hyperparamètres
from sklearn.metrics import accuracy_score, classification_report  # Pour évaluer les performances du modèle
from catboost import CatBoostClassifier  # Classifieur CatBoost

# Ignorer certains avertissements
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings('ignore')


In [46]:
df_tr = pd.read_csv(r'C:\Users\colin\Documents\ESEO\E5\docsPFE\creditcard_2023.csv')

In [47]:
# Séparation des données en caractéristiques (X) et cible (y)
X = df_tr.drop('Class', axis=1)
y = df_tr['Class']

In [48]:
# Définir les caractéristiques numériques (exclure les colonnes catégorielles)
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

In [49]:
# Définir le pipeline de transformation pour les colonnes numériques
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

StandardScaler() est un transformateur qui normalise les données en les centrant sur la moyenne 0 et en les réduisant à une échelle unitaire (écart-type 1). Cela est souvent nécessaire avant de passer à des algorithmes d'apprentissage automatique, car certains modèles sont sensibles à l'échelle des données (par exemple, les modèles basés sur la distance comme la régression logistique, SVM, etc.).

In [50]:
# Définir la transformation par colonnes
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)])

In [51]:
# Définir le modèle
model = Pipeline(steps=[('preprocessor', preprocessor),
                         ('classifier', CatBoostClassifier(verbose=False))])

In [52]:
# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test =train_test_split(X, y, test_size=0.2, random_state=42)


In [53]:
# Entraîner le modèle
model.fit(X_train, y_train)



Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['id', 'V1', 'V2', 'V3', 'V4',
                                                   'V5', 'V6', 'V7', 'V8', 'V9',
                                                   'V10', 'V11', 'V12', 'V13',
                                                   'V14', 'V15', 'V16', 'V17',
                                                   'V18', 'V19', 'V20', 'V21',
                                                   'V22', 'V23', 'V24', 'V25',
                                                   'V26', 'V27', 'V28',
                                                   'Amount'])])),
                ('classifier',
                 <catboost.core.CatBoostClassifier object at 0x000002281AF35240>)])

In [54]:
# Prédire sur l'ensemble de test
y_pred = model.predict(X_test)

In [55]:
# Calculer l'exactitude
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9996570705027874


In [56]:
# Afficher le rapport de classification avec couleurs et en-tête
from colorama import Fore, Back, Style
print("\n" + Back.BLUE + Fore.WHITE + "Classification Report" + Style.RESET_ALL)
report = classification_report(y_test, y_pred, output_dict=True)
for key, value in report.items():
    if key in ['0', '1']:  
        color = Fore.GREEN if value['precision'] > 0.8 else Fore.RED
        print(f"Class {key}:")
        print(f"  Precision: {color}{value['precision']:.2f}{Style.RESET_ALL}")
        color = Fore.GREEN if value['recall'] > 0.8 else Fore.RED
        print(f"  Recall: {color}{value['recall']:.2f}{Style.RESET_ALL}")
        color = Fore.GREEN if value['f1-score'] > 0.8 else Fore.RED
        print(f"  F1-score: {color}{value['f1-score']:.2f}{Style.RESET_ALL}")
        print(f"  Support: {value['support']}")
    else:
        print(key + ":", value)


Classification Report
Class 0:
  Precision: 1.00
  Recall: 1.00
  F1-score: 1.00
  Support: 56750.0
Class 1:
  Precision: 1.00
  Recall: 1.00
  F1-score: 1.00
  Support: 56976.0
accuracy: 0.9996570705027874
macro avg: {'precision': 0.9996569545708367, 'recall': 0.9996571914660413, 'f1-score': 0.9996570692311153, 'support': 113726.0}
weighted avg: {'precision': 0.999657078158671, 'recall': 0.9996570705027874, 'f1-score': 0.9996570705434346, 'support': 113726.0}
